In [13]:
import pandas as pd
import regex as re
import nltk
import preprocessor as p
import joblib
import tweepy
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
user_name = input("Please enter user name:")

Please enter user name: @trtworld


In [24]:
consumer_key = "qzWcxVXwd10Wqg9BfpAjAXEom"
consumer_secret = "kc4INOKZgZ6E7ojWWnbtqFFVx6V7InWqX1Lxgb6KY4zTVdciCw"
access_token = "323830961-Ejorvc54i7bx0nXtFsqrcvJ5oa4Nou3mDaE3TwI4"
access_token_secret = "SmvbwW6r3uxDDZ5nvNe8K63bXBfpIbRVDcbngXeCwi8zc"

auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [25]:
userID = user_name
user = api.get_user(userID)

new_tweets = api.user_timeline(screen_name = userID,count=200, tweet_mode="extended")
tweets = [[tweet.full_text] for tweet in new_tweets]
df = pd.DataFrame(tweets, columns=["tweet_text"])

In [26]:
def preprocessing(df):
    for i in range(len(df["tweet_text"])):
        df["tweet_text"][i] = re.sub(r'(?is)#', '', (df["tweet_text"][i]))
        df["tweet_text"][i] = p.clean(df["tweet_text"][i])
        df["tweet_text"][i] = re.sub(r'[^\w\s]','',df["tweet_text"][i])
        df.dropna(inplace=True)
    return df

In [27]:
def stop_words(df):
    stop_words = set(stopwords.words('english'))
    stop_words.update(('new','year', 'happy', 'first', 'one', 'lets', 'best', 'via', 'two', 'three', 'amp'))
    for i in range(len(df["tweet_text"])):
        word_tokens = word_tokenize(df["tweet_text"][i].lower())
        filtered_sentence = ""
        for w in word_tokens: 
            if w not in stop_words: 
                filtered_sentence += w
                filtered_sentence += " "
    filtered_sentence = filtered_sentence.strip()
    df["tweet_text"][i] = filtered_sentence
    df.dropna(inplace=True)
    return df

In [28]:
def stemming(df):
    for i in range(len(df["tweet_text"])):
        sentence = ""
        tweet = df["tweet_text"][i].lower().split()
        porter = nltk.SnowballStemmer('english')
        WNlemma = WordNetLemmatizer()
        for k in tweet:
            stemming = porter.stem(k)
            lem = WNlemma.lemmatize(stemming)
            sentence += lem
            sentence += " "
        sentence = sentence.strip()
        df["tweet_text"][i] = sentence
    return df

In [29]:
def vectorization(df):
    #vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2')
    #tfidf = pd.read_csv("preprocessed_data.csv")
    vectorizer = pickle.load(open("vector.pickel", "rb"))
    #loaded_vec = CountVectorizer(decode_error="replace",vocabulary=joblib.load("tf_idf.sav"))
    #tfidf = transformer.fit_transform(loaded_vec.fit_transform(df["tweet_text"]))
    #vectorizer.fit(tfidf["tweet_text"].values.astype('U'))
    x_test = vectorizer.transform(df)
    return x_test

In [30]:
df.head()

,tweet_text
0,2020 put Turkish drones in the spotlight as th...
1,British artist Damien Hirst to display more th...
2,A group of friends gifted their friend colourb...
3,Indonesian Sriwijaya Air plane with 62 people ...
4,Tehran bans foreign companies from testing Cov...


In [31]:
for i in range(len(df["tweet_text"])):
    if df["tweet_text"][i] == '':
        df.drop([i], inplace=True)
df = df.reset_index(drop=True)

In [35]:
df = df.applymap(str.lower)
pre_df = preprocessing(df)
final_df = stop_words(pre_df)
df = stemming(final_df)
#df = vectorization(df)

In [38]:
models = []
with open("models.pckl", "rb") as f:
    while True:
        try:
            models.append(pickle.load(f))
        except EOFError:
            break

In [54]:
categories = ['art', 'game', 'movie', 'music', 'politic', 'science', 'sport', 'technology', 'travel', 'TV']
total = []
for i in range(len(df["tweet_text"])):
    test = [df["tweet_text"][i]]
    test = vectorization(test)
    result = []
    final = []
    for model in models:
        result.append(model.predict_proba(test))
    for k in range(len(result)):
        final.append(result[k][0][1])
    index = final.index(max(final))
    total.append(categories[index])

In [67]:
data = {}
data.setdefault("result_name", [])
data.setdefault("result_percentage", [])

for category in categories:
    percentage = int(total.count(category) * 100 / len(total))
    data["result_name"].append(category)
    data["result_percentage"].append(percentage)